In [68]:
import streamlit as st
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import folium
import zipfile
import os

st.set_page_config(page_title="ENEM Analysis",
                   page_icon=":bar_chart:",
                   layout="wide")

In [7]:
import json

# Abra o arquivo JSON para leitura
with open('data_dictionaries.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Agora, 'data' contém o conteúdo do JSON
print(data.keys())

dict_keys(['columns', 'renamed_columns', 'faixa_etaria', 'estado_civil', 'etnia', 'nacionalidade', 'situacao_ensino_medio', 'tipo_instituicao', 'treineiro', 'situacao_redacao', 'status_internet', 'faixa_renda'])


In [6]:
# Arrays e dicionários úteis

columns = ["NU_ANO","TP_NACIONALIDADE","TP_SEXO","TP_FAIXA_ETARIA","TP_COR_RACA","TP_ESTADO_CIVIL",
           "TP_ST_CONCLUSAO","TP_ESCOLA","IN_TREINEIRO","NO_MUNICIPIO_ESC","SG_UF_PROVA","TP_PRESENCA_CN",
           "TP_PRESENCA_CH","TP_PRESENCA_LC","TP_PRESENCA_MT","NU_NOTA_CN","NU_NOTA_CH","NU_NOTA_LC",
           "NU_NOTA_MT","TP_STATUS_REDACAO","NU_NOTA_COMP1","NU_NOTA_COMP2","NU_NOTA_COMP3","NU_NOTA_COMP4",
           "NU_NOTA_COMP5","NU_NOTA_REDACAO","TP_LINGUA","Q006","Q025"]

renamed_columns = ["Ano","Nacionalidade","Sexo","Idade","Cor","Estado_civil","Situacao_EM","Tipo_Escola",
                   "Treineiro","Municipio","Estado","Presenca_CN","Presenca_CH","Presenca_LC","Presenca_MT",
                   "Ciencias_Natureza","Ciencias_Humanas","Linguagens_Codigos","Matematica","Status_Redacao",
                   "Ortografia","Desenvolvimento","Informacoes","Organizacao","Proposta","Redacao","Lingua_Estrangeira","Renda","Internet"]

# Conseguindo a relação das categorias das colunas
faixa_etaria = {
    1: "Menor de 17 anos",
    2: "17 anos",
    3: "18 anos",
    4: "19 anos",
    5: "20 anos",
    6: "21 anos",
    7: "22 anos",
    8: "23 anos",
    9: "24 anos",
    10: "25 anos",
    11: "Entre 26 e 30 anos",
    12: "Entre 31 e 35 anos",
    13: "Entre 36 e 40 anos",
    14: "Entre 41 e 45 anos",
    15: "Entre 46 e 50 anos",
    16: "Entre 51 e 55 anos",
    17: "Entre 56 e 60 anos",
    18: "Entre 61 e 65 anos",
    19: "Entre 66 e 70 anos",
    20: "Maior de 70 anos"
}

estado_civil = {
    0: "Não informado",
    1: "Solteiro(a)",
    2: "Casado(a)/Mora com companheiro(a)",
    3: "Divorciado(a)/Desquitado(a)/Separado(a)",
    4: "Viúvo(a)"
}

etnia = {
    0: "Não declarado",
    1: "Branca",
    2: "Preta",
    3: "Parda",
    4: "Amarela",
    5: "Indígena",
    6: "Não dispõe da informação"
}

nacionalidade = {
    0: "Não informado",
    1: "Brasileiro(a)",
    2: "Brasileiro(a) Naturalizado(a)",
    3: "Estrangeiro(a)",
    4: "Brasileiro(a) Nato(a), nascido(a) no exterior"
}

situacao_ensino_medio = {
    1: "Já concluí o Ensino Médio",
    2: "Estou cursando e concluirei o Ensino Médio em 2022",
    3: "Estou cursando e concluirei o Ensino Médio após 2022",
    4: "Não concluí e não estou cursando o Ensino Médio"
}

tipo_instituicao = {
    1: "Não Respondeu",
    2: "Pública",
    3: "Privada",
    4: "Exterior"
}

treineiro = {
    1: "Sim",
    0: "Não"
}

situacao_redacao = {
    1: "Sem problemas",
    2: "Anulada",
    3: "Cópia Texto Motivador",
    4: "Em Branco",
    5:"Fere Direitos Humanos",
    6: "Fuga ao tema",
    7: "Não atendimento ao tipo textual",
    8: "Texto insuficiente",
    9: "Parte desconectada"
}

status_internet = {
    'A': 'Não',
    'B': 'Sim'
}

faixa_renda = {
    'A': 'Nenhuma Renda',
    'B': 'Até R$ 1.212,00',
    'C': 'De R$ 1.212,01 até R$ 1.818,00',
    'D': 'De R$ 1.818,01 até R$ 2.424,00',
    'E': 'De R$ 2.424,01 até R$ 3.030,00',
    'F': 'De R$ 3.030,01 até R$ 3.636,00',
    'G': 'De R$ 3.636,01 até R$ 4.848,00',
    'H': 'De R$ 4.848,01 até R$ 6.060,00',
    'I': 'De R$ 6.060,01 até R$ 7.272,00',
    'J': 'De R$ 7.272,01 até R$ 8.484,00',
    'K': 'De R$ 8.484,01 até R$ 9.696,00',
    'L': 'De R$ 9.696,01 até R$ 10.908,00',
    'M': 'De R$ 10.908,01 até R$ 12.120,00',
    'N': 'De R$ 12.120,01 até R$ 14.544,00',
    'O': 'De R$ 14.544,01 até R$ 18.180,00',
    'P': 'De R$ 18.180,01 até R$ 24.240,00',
    'Q': 'Acima de R$ 24.240,00'
}

NU_ANO
TP_NACIONALIDADE
TP_SEXO
TP_FAIXA_ETARIA
TP_COR_RACA
TP_ESTADO_CIVIL
TP_ST_CONCLUSAO
TP_ESCOLA
IN_TREINEIRO
NO_MUNICIPIO_ESC
SG_UF_PROVA
TP_PRESENCA_CN
TP_PRESENCA_CH
TP_PRESENCA_LC
TP_PRESENCA_MT
NU_NOTA_CN
NU_NOTA_CH
NU_NOTA_LC
NU_NOTA_MT
TP_STATUS_REDACAO
NU_NOTA_COMP1
NU_NOTA_COMP2
NU_NOTA_COMP3
NU_NOTA_COMP4
NU_NOTA_COMP5
NU_NOTA_REDACAO
TP_LINGUA
Q006
Q025


In [69]:
# Configurações do Pandas
pd.options.display.max_columns = 26

In [70]:
def build_dataframe():
    # Arrays e dicionários úteis
    columns = ["NU_ANO","TP_NACIONALIDADE","TP_SEXO","TP_FAIXA_ETARIA","TP_COR_RACA","TP_ESTADO_CIVIL",
            "TP_ST_CONCLUSAO","TP_ESCOLA","IN_TREINEIRO","NO_MUNICIPIO_ESC","SG_UF_PROVA","TP_PRESENCA_CN",
            "TP_PRESENCA_CH","TP_PRESENCA_LC","TP_PRESENCA_MT","NU_NOTA_CN","NU_NOTA_CH","NU_NOTA_LC",
            "NU_NOTA_MT","TP_STATUS_REDACAO","NU_NOTA_COMP1","NU_NOTA_COMP2","NU_NOTA_COMP3","NU_NOTA_COMP4",
            "NU_NOTA_COMP5","NU_NOTA_REDACAO"]

    renamed_columns = ["Ano","Nacionalidade","Sexo","Idade","Cor","Estado_civil","Situacao_EM","Tipo_Escola",
                    "Treineiro","Municipio","Estado","Presenca_CN","Presenca_CH","Presenca_LC","Presenca_MT",
                    "Ciencias_Natureza","Ciencias_Humanas","Linguagens_Codigos","Matematica","Status_Redacao",
                    "Ortografia","Desenvolvimento","Informacoes","Organizacao","Proposta","Redacao"]
    
    zip_data = {
    "Zip Files":[],
    "Folder Data":  os.listdir("information")
    }

    for zippedArch in zip_data["Folder Data"]:
        with zipfile.ZipFile(f"information\{zippedArch}") as zippedData:
            for file in zippedData.namelist():
                if "microdados" in file.lower() and file.endswith(".csv"):
                    zip_data["Zip Files"].append(file)
                    
    # Lambda pra ordenar os arquivos do mais antigo para o mais novo 2015 - 2022
    zip_data["Zip Files"].sort(key = lambda x: x[22:-4])
    zip_data["Folder Data"].sort(key = lambda x: x[16:-4])
    
    # Pegando os ultimos 3 anos
    zip_data["Folder Data"] = zip_data["Folder Data"][-3:]
    zip_data["Zip Files"] = zip_data["Zip Files"][-3:]

    # Criando o megadataframe filtrado
    enem_collection = []
    for i in range(len(zip_data["Zip Files"])):
        with zipfile.ZipFile(f"information\{zip_data['Folder Data'][i]}") as microdata:
            with microdata.open(zip_data["Zip Files"][i]) as csv:
                
                data = pd.read_csv(csv,encoding="ISO-8859-1",sep=";",usecols=columns)
                # Renomeando as colunas
                data.columns = renamed_columns
                # Removendo as linhas onde não temos notas
                data.dropna(subset=["Ciencias_Natureza",'Ciencias_Humanas',"Ciencias_Humanas","Matematica","Redacao"],inplace=True)
                data.reset_index(inplace=True,drop=True)
                # Adicionando no array pra criar o Dataframe completo posteriormente
                enem_collection.append(data)
                
    return pd.concat(enem_collection)
    

In [71]:
enem = build_dataframe()


In [72]:
# Calculando a Nota Total de cada pessoa
enem['Nota Total'] = enem[['Ciencias_Natureza','Ciencias_Humanas','Matematica','Linguagens_Codigos','Redacao']].sum(axis=1)/5

In [73]:
enem.head()

,Ano,Nacionalidade,Sexo,Idade,Cor,Estado_civil,Situacao_EM,Tipo_Escola,Treineiro,Municipio,Estado,Presenca_CN,Presenca_CH,...,Presenca_MT,Ciencias_Natureza,Ciencias_Humanas,Linguagens_Codigos,Matematica,Status_Redacao,Ortografia,Desenvolvimento,Informacoes,Organizacao,Proposta,Redacao,Nota Total
0,2020,11,M,2,3,1,1,1,0,NaN,RN,1,1,...,1,604.1,661.7,595.3,711.3,1.0,120.0,120.0,120.0,120.0,100.0,580.0,630.48
1,2020,2,M,1,3,1,2,2,0,Santana de Parnaíba,SP,1,1,...,1,620.8,675.0,624.2,759.4,1.0,140.0,200.0,140.0,120.0,160.0,760.0,687.88
2,2020,3,F,1,3,1,1,1,0,NaN,RS,1,1,...,1,498.1,604.7,505.4,526.7,1.0,140.0,120.0,140.0,140.0,160.0,700.0,566.98
3,2020,4,M,1,1,1,1,1,0,NaN,SP,1,1,...,1,604.6,604.8,562.1,753.2,1.0,140.0,120.0,120.0,120.0,100.0,600.0,624.94
4,2020,5,F,2,3,1,1,1,0,NaN,PB,1,1,...,1,439.7,383.5,486.2,448.5,1.0,120.0,120.0,120.0,140.0,100.0,600.0,471.58


In [74]:
def main():
    # Configuração inicial do Streamlit
    st.title("Mapa do Brasil usando Folium")
    st.sidebar.header("Configurações do Mapa")

    # Configurações do mapa
    zoom_level = st.sidebar.slider("Selecione o nível de zoom", 5, 15, 5)
    map_width = st.sidebar.slider("Selecione a largura do mapa", 500, 1500, 1000)

    # Criar o mapa do Brasil usando Folium
    mapa_brasil = folium.Map(location=[-15.788497, -47.879873], zoom_start=zoom_level, control_scale=True)

    # Adicionar limites do Brasil
    folium.GeoJson(
        "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson",
        name="geojson"
    ).add_to(mapa_brasil)

    # Renderizar o mapa no Streamlit
    folium_static(mapa_brasil, width=map_width)

def folium_static(fig, width=700, height=500):
    """Função para renderizar o mapa Folium no Streamlit."""
    fig.add_child(folium.LatLngPopup())  # Adicionar popup de coordenadas
    fig = fig._repr_html_().replace("width: 100%;", f"width: {width}px; height: {height}px;")
    st.write(folium.Html(fig, width=width, height=height, script=True))


In [86]:
def get_states_quality(df:pd.DataFrame):
    cloned = df.copy()
    
    # Filtrando os alunos por nota 800+ com uma coluna auxiliar
    cloned['Great_student'] = cloned['Nota Total'].apply(lambda x: "Great" if x >= 800 else "Bad")
    
    # Conseguindo os valores como porcentagem e dropando a coluna auxiliar
    brazil_grades = pd.DataFrame(cloned.groupby('Estado').value_counts(normalize=True)).reset_index()
    brazil_grades = pd.DataFrame(cloned.groupby('Estado')['Great_student'].value_counts(normalize=True)).reset_index()
    
    # Pivotando a tabela e organizando os dados obtidos
    brazil_grades = brazil_grades.pivot_table('proportion', ['Estado'], 'Great_student').reset_index()

    # Elevando pra ficar em porcentagem de fato
    # brazil_grades = brazil_grades.map(lambda x: x*100 if type(x) == float else x)
    
    return brazil_grades

In [87]:
data = get_states_quality(enem)

In [88]:
data

Great_student,Estado,Bad,Great
0,AC,1.000000,NaN
1,AL,0.999768,0.000232
2,AM,0.999926,0.000074
3,AP,0.999979,0.000021
4,BA,0.999738,0.000262
5,CE,0.999317,0.000683
6,DF,0.998251,0.001749
7,ES,0.999067,0.000933
8,GO,0.999054,0.000946
9,MA,0.999830,0.000170
